In [ ]:
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.5 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback,
    pipeline

)

from tqdm import tqdm
import glob
import json
import os
import re

In [ ]:
# model_checkpoints = "/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint2/KoBART_Summary_v2/checkpoint-4376"


# tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
def data_mining(data):
  body_list = []
  text_list = []
  total_list = []
  with open(data) as f:
        data = json.load(f)
  for datum in data["data"]:
      body_list.append(datum['body'])  
  for text in body_list:
    text_list.append(text)
  for utt in text_list:
      utt_list = []
      for i in range(len(utt)):
          utt_list.append(utt[i]['utterance'])
      total_list.append(utt_list)
  return total_list[:11]

def data_load(filename):
    text = []

    for file in tqdm(filename):
      total_list = data_mining(file)
      for data in total_list:
        text.append(data)

    return text


In [ ]:
dirname = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_train"
filenames = os.listdir(dirname) 
train_full_filename = []

for filename in filenames:
    fn = os.path.join(dirname, filename)
    if dirname + '/.ipynb_checkpoints' != fn:
        train_full_filename.append(fn)

dirname2 = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_valid"
filenames2 = os.listdir(dirname2) 
val_full_filename = []

for filename in filenames2:
    fn2 = os.path.join(dirname2, filename)
    if dirname + '/.ipynb_checkpoints' != fn2:
        val_full_filename.append(fn2)

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub(r".+", '', sentence)
    sentence = re.sub("[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [ ]:
test_dataset = data_load(val_full_filename)
test_dataset = data_process(test_dataset)


  0%|          | 0/99 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
gen_kwargs = {"length_penalty": 2.0, "num_beams":8, "max_length": 128}

# `haesun`를 자신의 허브 사용자 이름으로 바꾸세요.
pipe = pipeline("summarization", model="jx7789/kobart_summary")

In [ ]:
for dialogue in test_dataset:
  print(pipe("[sep]".join(dialogue), **gen_kwargs)[0]["summary_text"])